In [1]:
import os
import numpy as np
#import keras
import tensorflow
import glob
#from tensorflow.keras.utils import np_utils
from PIL import Image
from sklearn.model_selection import StratifiedKFold


from tensorflow.keras.applications import ResNet152V2
#from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Dense,Input,GlobalAveragePooling2D,Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, KFold
# -------------------------------------------------------------------------------------
#                        初期設定部
# -------------------------------------------------------------------------------------

# GrayScaleのときに1、COLORのときに3にする
COLOR_CHANNEL = 3

# 入力画像サイズ(画像サイズは正方形とする)
INPUT_IMAGE_SIZE = 224

# 訓練時のバッチサイズとエポック数
BATCH_SIZE = 16
EPOCH_NUM = 350

# 使用する訓練画像の入ったフォルダ(ルート)
TRAIN_PATH =  
TEST_PATH = 

# 使用する訓練画像の各クラスのフォルダ名
folder = os.listdir(TRAIN_PATH)
test_folder = os.listdir(TEST_PATH)

# CLASS数を取得する
CLASS_NUM = len(folder)
print("クラス数 : " + str(CLASS_NUM))

#validationの割合
validation_rate = 0.2

# 分割数を設定
k = 5

#分割交差検証の平均算出用
test_loss = [0]*k
test_accuracy = [0]*k
sum_accuracy = 0
sum_val_accuracy = 0
sum_test_accuracy = 0
sum_loss = 0
sum_val_loss = 0
sum_test_loss = 0

クラス数 : 80


In [ ]:
# -------------------------------------------------------------------------------------
#                        訓練画像入力部
# -------------------------------------------------------------------------------------

# 各フォルダの画像を読み込む
v_image = []
v_label = []
for index, name in enumerate(folder):
    dir = TRAIN_PATH + "\\" + name
    files = glob.glob(dir + "\\*.jpg")
    print(dir)
    for i, file in enumerate(files):
        if COLOR_CHANNEL == 1:
            img = load_img(file, color_mode = "grayscale", target_size=(INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE))
        elif COLOR_CHANNEL == 3:
            img = load_img(file, color_mode = "rgb", target_size=(INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE))
        array = img_to_array(img)
        v_image.append(array)
        v_label.append(index)

v_image = np.array(v_image)
v_label = np.array(v_label)
#print(v_image)
#print(v_label)

# testフォルダの画像を読み込む
t_image = []
t_label = []
for index, name in enumerate(test_folder):
    dir = TEST_PATH + "\\" + name
    testfiles = glob.glob(dir + "\\*.jpg")
    print(dir)
    for i, file in enumerate(testfiles):
        if COLOR_CHANNEL == 1:
            img = load_img(file, color_mode = "grayscale", target_size=(INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE))
        elif COLOR_CHANNEL == 3:
            img = load_img(file, color_mode = "rgb", target_size=(INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE))
        array = img_to_array(img)
        t_image.append(array)
        t_label.append(index)

t_image = np.array(t_image)
t_label = np.array(t_label)
#print(t_image)
#print(t_label)

# imageの画素値をint型からfloat型にする
v_image = v_image.astype('float32')
t_image = t_image.astype('float32')
# 画素値を[0～255]⇒[0～1]とする
v_image = v_image / 255.0
t_image = t_image / 255.0

# データの前処理
# ラベルを変換
label_binarizer = LabelBinarizer()
#v_label = label_binarizer.fit_transform(v_label)

#StratifiedKFoldの設定
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

In [3]:
#testデータの処理
test_data = t_image
test_label = label_binarizer.fit_transform(t_label)



In [ ]:
for fold_num, (train_index, val_index) in enumerate(skf.split(v_image, v_label)):
    print(f"Fold {fold_num+1}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={val_index}")
    # 訓練データとラベル
    train_data, train_label = v_image[train_index], v_label[train_index]
    
    # テストデータとラベル
    val_data, val_label = v_image[val_index], v_label[val_index]
    
    #labelbinarizer
    train_label = label_binarizer.fit_transform(train_label)
    val_label = label_binarizer.fit_transform(val_label)
    
    Resnet_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in Resnet_model.layers:
        layer.trainable = False
    
    x = Resnet_model.output
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(units=CLASS_NUM, activation='softmax')(x)
    
    model = Model(inputs=Resnet_model.input,outputs=outputs)
    
    optimizer = Adam
    # モデルのコンパイル
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    # 訓練
    history = model.fit(train_data, train_label, validation_data = (val_data,val_label), batch_size = BATCH_SIZE, epochs=EPOCH_NUM)
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    Accuracy = history.history['accuracy']
    val_Accuracy = history.history['val_accuracy']
    epochs = range(1, len(loss) + 1)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.yscale('log')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(epochs, Accuracy, 'b', label='Training Accuracy')
    plt.plot(epochs, val_Accuracy, 'r', label='Validation Accuracy')
    plt.yscale('log')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
    #print(Accuracy[-1])
    

    # 訓練データの予測
    test_predictions = model.predict(test_data)
    # 予測結果からクラスを選択
    test_predicted_classes = np.argmax(test_predictions, axis=1)
    # 正解ラベルをクラスに戻す
    test_true_classes = np.argmax(test_label, axis=1)
    # 混同行列を計算
    confusion = confusion_matrix(test_true_classes, test_predicted_classes)
    # 混同行列を可視化
    display = ConfusionMatrixDisplay(confusion, display_labels=folder)
    display.plot(cmap=plt.cm.Blues, values_format="d")
    plt.title("Confusion Matrix (Training Data)")
    plt.show()
    
    score = model.evaluate(test_data, test_label, verbose=0)
    print(len(test_data))
    test_loss[fold_num]=score[0]
    test_accuracy[fold_num]=score[1]
    print('Loss:', score[0])
    print('Accuracy:', score[1])
    
    sum_accuracy = sum_accuracy + Accuracy[-1]
    sum_val_accuracy = sum_val_accuracy + val_Accuracy[-1]
    sum_test_accuracy = sum_test_accuracy + score[1]
    sum_loss = sum_loss + loss[-1]
    sum_val_loss = sum_val_loss + val_loss[-1]
    sum_test_loss = sum_test_loss + score[0]
        
ave_accuracy = sum_accuracy/k
ave_val_accuracy = sum_val_accuracy/k
ave_test_accuracy = sum_test_accuracy/k
ave_loss = sum_loss/k
ave_val_loss = sum_val_loss/k
ave_test_loss = sum_test_loss/k

for j in range(k):
    print(f"hold{j+1}_test_loss:{test_loss[j]}")

    
for j in range(k):
    print(f"hold{j+1}_test_accuracy:{test_accuracy[j]}")
    

print(f"平均accuracy: {ave_accuracy}")
print(f"平均val_accuracy: {ave_val_accuracy}")
print(f"平均test_accuracy: {ave_test_accuracy}")
print(f"平均loss:{ave_loss}")
print(f"平均val_loss:{ave_val_loss}")
print(f"平均test_loss:{ave_test_loss}")



In [ ]:
from sklearn.metrics import classification_report

names = ['Bear', 'Brown bear', 'Bull', 'Butterfly', 'Camel', 'Canary', 'Caterpillar', 'Cattle', 'Centipede', 'Cheetah', 'Chicken', 'Crab', 'Crocodile', 'Deer', 'Duck', 'Eagle', 'Elephant', 'Fish', 'Fox', 'Frog', 'Giraffe', 'Goat', 'Goldfish', 'Goose', 'Hamster', 'Harbor seal', 'Hedgehog', 'Hippopotamus', 'Horse', 'Jaguar', 'Jellyfish', 'Kangaroo', 'Koala', 'Ladybug', 'Leopard', 'Lion', 'Lizard', 'Lynx', 'Magpie', 'Monkey', 'Moths and butterflies', 'Mouse', 'Mule', 'Ostrich', 'Otter', 'Owl', 'Panda', 'Parrot', 'Penguin', 'Pig', 'Polar bear', 'Rabbit', 'Raccoon', 'Raven', 'Red panda', 'Rhinoceros', 'Scorpion', 'Sea lion', 'Sea turtle', 'Seahorse', 'Shark', 'Sheep', 'Shrimp', 'Snail', 'Snake', 'Sparrow', 'Spider', 'Squid', 'Squirrel', 'Starfish', 'Swan', 'Tick', 'Tiger', 'Tortoise', 'Turkey', 'Turtle', 'Whale', 'Woodpecker', 'Worm', 'Zebra']
report = classification_report(test_true_classes, test_predicted_classes,target_names=names)
print(report)